In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
trainset = np.load('small_train.npy')
testset = np.load('small_test.npy')

In [3]:
np.any(np.isnan(trainset))

False

In [4]:
def dict_from_array(arr):
    d = {}
    for row in range(arr.shape[1]):
        d[str(row)] = arr[:,row]
    return d

def no_zeros(arr):
    arr[arr==0] = 1e-8
    return arr

In [5]:
#Taken from https://www.tensorflow.org/get_started/datasets_quickstart
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size=30000).repeat(count=None).batch(batch_size)

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()


In [6]:
train_x = dict_from_array(no_zeros(trainset[:,:-1]))
train_y = trainset[:,-1]
test_x = dict_from_array(no_zeros(testset[:,:-1]))
test_y = testset[:,-1]
print(len(train_y))

28458


In [7]:
feature_columns = []
for i in range(len(train_x)):
    feature_columns.append(tf.feature_column.numeric_column(key=str(i)))

In [8]:
feature_columns

[_NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='10', shape=(1,), de

In [9]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [5,5],
    n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Micah\\AppData\\Local\\Temp\\tmpj5jvvjxq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F4104080F0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
classifier.train(
    input_fn=lambda:train_input_fn(train_x,train_y,100), steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\Micah\AppData\Local\Temp\tmpj5jvvjxq\model.ckpt-5
INFO:tensorflow:Saving checkpoints for 6 into C:\Users\Micah\AppData\Local\Temp\tmpj5jvvjxq\model.ckpt.
INFO:tensorflow:loss = 69.70061, step = 6
INFO:tensorflow:Saving checkpoints for 105 into C:\Users\Micah\AppData\Local\Temp\tmpj5jvvjxq\model.ckpt.
INFO:tensorflow:Loss for final step: 69.64908.


In [14]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 100))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-02-06-07:22:56
INFO:tensorflow:Restoring parameters from C:\Users\Micah\AppData\Local\Temp\tmpj5jvvjxq\model.ckpt-105
INFO:tensorflow:Finished evaluation at 2018-02-06-07:22:59
INFO:tensorflow:Saving dict for global step 105: accuracy = 0.5341556, accuracy_baseline = 0.5294118, auc = 0.5239077, auc_precision_recall = 0.5518825, average_loss = 0.69018704, global_step = 105, label/mean = 0.5294118, loss = 68.199104, prediction/mean = 0.5256007

Test set accuracy: 0.534

